In [8]:

import os
import pandas as pd
import numpy as np

class SessionHnd:
  def __init__(self, data_path):
    self.data_path = data_path
    self.sessions=pd.read_hdf('%s/ecephys_sessions.h5' % data_path)
    self._set_session_ids()

  def _set_session_ids(self):
    session_ids=[]
    for filename in os.listdir(self.data_path):
      if filename.startswith('1'):
        session_ids.append(int(filename[0:10]))
    session_ids=list(set(session_ids))
    self.session_ids = session_ids

  def get_acronyms_for_session(self, session_id):
    if session_id not in self.session_ids:
        raise KeyError("A session_id '{}' não foi encontrada.".format(session_id))

    acronyms=self.sessions.loc[session_id].structure_acronyms
    acronyms=acronyms.split("'")[1:-1]
    while (', ' in acronyms):
      acronyms.remove(', ')
    acronyms.remove('root') # Remove root
    acronyms = [item for item in acronyms if not item.startswith('DG-')]
    acronyms.append('DG')
    acronyms.sort()
    return (acronyms)

  def get_sessions_for (self, acronym):

    sessions = []
    for session_id in self.session_ids:
      if (self.exists_session_for(session_id, acronym)):
        sessions.append(session_id)
    return (sessions)

  def exists_session_for(self, session_id, acronym):
    filename = '%s/%d-%s-spk.h5' % (self.data_path, session_id, acronym)
    return (os.path.exists(filename))

  def get_spikes (self, session_id, acronym):
    spikes=None

    filename_spk='%s/%s-%s-spk.h5' % (self.data_path,session_id,acronym)
    if not os.path.exists(filename_spk):
        return (spikes)

    spikes = pd.read_hdf(filename_spk, key='spk')
    spikes.name = '%s-%s' % (session_id,acronym)
    spikes=spikes.sort_index()
    return (spikes)

  def get_speed_run (self, session_id):
    filename = '%s/behavior/%d-comp-vel.h5' % (self.data_path,session_id)
    if not os.path.exists(filename):
        raise FileNotFoundError("File not found '{}'.".format(filename))
    v=pd.read_hdf (filename)
    return (v)

  def get_reward (self, session_id):
    filename = '%s/%d-rew.h5' % (self.data_path,session_id)
    reward = pd.read_hdf(filename)
    return (reward)

  def get_stim (self, session_id):
    filename = '%s/%d-stim.h5' % (self.data_path,session_id)
    stim = pd.read_hdf(filename)
    return (stim)

  def get_stim_times (self, stim_name, session_id):
    stim = self.get_stim (session_id)
    times = None
    stim_names = {'natural': 'Nat', 'spont': 'spont', 'gabor': 'gabor', 'flash': 'flash'}
    if (stim_name in stim_names.keys()):
      N=stim.stimulus_name.str.startswith(stim_names[stim_name])
      times=stim[N]['start_time'].dropna().values
    return (times)


In [9]:
data_path='/home/napvasconcelos/data/allen-icvb'
shnd = SessionHnd (data_path=data_path)

In [10]:
spikes = shnd.get_spikes(session_id=1044594870, acronym='VISl')

In [23]:
ms=1e-3
neurons=spikes.unique()
nrows=len(neurons)
t=spikes.index
bin_size = 10*ms
bins=np.arange(0,spikes.index[-1],bin_size)
count, tbins=np.histogram(t,bins)
ncols = len(count)
print (count.shape)
B=np.zeros((nrows, ncols), dtype=np.int8)
i=0
for i, n in enumerate(neurons):
    B[i],_ =  np.histogram(spikes[spikes==n],bins)



(972014,)


In [24]:
i

131

In [21]:
B[:,100:110]

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int8)

In [ ]:
def get_neuronal_pattern (self, tsamples, B, width):
    nneurons=B.shape[0]

    nbins = width
    pattern = np.zeros((nneurons,nbins),dtype=np.int8)
    for s0  in tsamples:
        #self.logger.info ('get pattern %d ' % i)
        mask = (spikes==unit_id)
        s= spikes[mask].index.astype(float)
        spk=s[(s>=t0) & (s<t1)]
        pattern[i]=np.histogram(spk-t0, bins)[0]
        i+=1

    return (pattern)